## Hands-On Data Preprocessing in Python
Learn how to effectively prepare data for successful data analytics

## Data Cleaning Level Ⅱ - Unpack, restructure, and reformulate the table

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Example 2.1 – unpacking columns & reformulating the table

In [ ]:
from os import listdir
directory = 'Speeches'

FileNames = listdir(directory)
speech_df = pd.DataFrame(index=range(len(FileNames)),columns=['File Name','The Content'])

for i,f_name in enumerate(FileNames):
  f = open(directory + "/" + f_name, "r", encoding='utf-8')
  f_content = f.readlines()
  f.close()

  speech_df.at[i,'File Name'] = f_name
  speech_df.at[i,'The Content'] = f_content[0]

speech_df.columns = ['FileName','Content']

In [ ]:
speech_df

The *FileName* column contains the following information about the speeches in the dataset:
*   City: The city where the speech was given
*   Date: The date when the speech was given
*   Year: The year when the speech was given
*   Month: The month when the speech was given
*   Day: The day when the speech was given


### Unpacking FileName
---
The following are the steps we need to take for the unpacking process:
1. Extract City: Use Month from the **CitynameMonthDD_YYYY**.txt pattern to extract
the city. Based on this pattern, everything that comes before Month is Cityname.
2. Extract Date: Use the extracted Cityname to extract Date.
3. Extract Year, Month, and Day from Date.


In [ ]:
Months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Oct','Sep','Nov','Dec']
def SeparateCity(v):
  for mon in Months:
    if (mon in v):
      return v[:v.find(mon)]

speech_df['City'] = speech_df.FileName.apply(SeparateCity)

In [ ]:
speech_df.head()

In [ ]:
def SeparateDate(r):
  return r.FileName[len(r.City):r.FileName.find('.txt')]

speech_df['Date'] = speech_df.apply(SeparateDate,axis=1)
speech_df.Date = pd.to_datetime(speech_df.Date,format='%b%d_%Y')

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
speech_df.head()

In [ ]:
def extractDMY(r):
  r['Day'] = r.Date.day
  r['Month'] = r.Date.month
  r['Year'] = r.Date.year
  return r

speech_df = speech_df.apply(extractDMY,axis=1)

In [ ]:
speech_df.drop(columns=['FileName'],inplace=True)

In [ ]:
speech_df.head()

### Unpacking Content
---
The following code creates the *FindWordRatio()* function and applies it to **speech_df**. The function uses a **for** loop to add four new columns to the DataFrame, one column for each of the four words. The calculation for each word is simple: the returning value for each word is the total occurrence of the word in the speech *(row.Content.count(w))*, divided by the total number of words in the speech *(total_n_words)*:

In [ ]:
Words = ['vote','tax','campaign','economy']

def FindWordRatio(row):
  total_n_words = len(row.Content.split(' '))
  for w in Words:
    row['r_{}'.format(w)] = row.Content.count(w)/total_n_words
  return row

speech_df = speech_df.apply(FindWordRatio,axis=1)

In [ ]:
speech_df.head()

### Reformulate a new table for visualization
---
When we need to reformulate a dataset so that its new definition of data objects is an aggregation of the current definition of data objects, we need to perform two steps:
1. Create a column that can be the unique identifier for the reformulated dataset.
2. Use a function that can reformulate the dataset while applying the aggregate functions. The pandas functions that can do this are <font color='blue'>.groupby()</font> and <font color='blue'>.pivot_table()</font>.

In [ ]:
Months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Oct','Sep','Nov','Dec']
lambda_func = lambda r: '{}_{}'.format(r.Year,Months[r.Month-1])
speech_df['Y_M'] = speech_df.apply(lambda_func,axis=1)

> **Lambda functions** are similar to user-defined functions but without a name. They're commonly referred to as anonymous functions. Lambda functions are efficient whenever you want to create a function that will only contain simple expressions – that is, expressions that are usually a single line of a statement. They're also useful when you want to use the function once.

In [ ]:
speech_df.head()

In [ ]:
Words = ['vote','tax','campaign','economy']
vis_df = speech_df.pivot_table(
    index= ['Y_M'],
    values= ['r_{}'.format(w) for w in Words],
    aggfunc= np.mean)

# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [ ]:
vis_df

In [ ]:
vis_df = pd.DataFrame({
  'r_vote': speech_df.groupby('Y_M').r_vote.mean(),
  'r_tax': speech_df.groupby('Y_M').r_tax.mean(),
  'r_campaign': speech_df.groupby('Y_M').r_campaign.mean(),
  'r_economy': speech_df.groupby('Y_M').r_economy.mean()
})

In [ ]:
vis_df

### The last step: draw the visualization
---
The preceding code creates two lists: **column_order** and **row_order**. As their names suggest, these lists are the order in which the columns and rows will be shown on the visual. The **column_order** is the list of words based on the summation of their occurrence ratio, while **row_order** is the list of *Y_M* based on their natural order in the calendar.

In [ ]:
column_order = vis_df.sum().sort_values(ascending=False).index
row_order = speech_df.sort_values('Date').Y_M.unique()

vis_df[column_order].loc[row_order].plot.bar(figsize=(10,4))
plt.legend(['vote','tax','campaign','economy'],ncol=2)
plt.xlabel('Year_Month')
plt.ylabel('Average Word Frequency')
plt.show()

### Example 2.2 – restructure the table

In this example, we will use the **Customer Churn.csv** dataset. This dataset contains the records of 3,150 customers of a telecommunication company. The dataset also specifies whether each customer was churned or not 3 months after the 9 months of collecting the activity data of the customers. Customer churning, from a telecommunication company's point of view, means the customer stops using the company's services and receives the services from the company's competition.

We would like to use box plots to compare the two populations of churning customers and non-churning customers for the following activity columns: **Call Failure, Subscription Length, Seconds of Use, Frequency of use, Frequency of SMS, and Distinct Called Numbers.**

In [ ]:
customer_df = pd.read_csv('Customer Churn.csv')
customer_df.head(1)

> While the column titles are intuitive, they can become more codable. The following line of code makes sure that the columns are also codable:

In [ ]:
customer_df.columns = ['Call_Failure', 'Complains', 'Subscription_Length', 'Seconds_of_Use',
                       'Frequency_of_use', 'Frequency_of_SMS', 'Distinct_Called_Numbers',
                       'Status', 'Churn']

In [ ]:
customer_df.head(5)

> For the **box plot** we want to draw, the dictionary we need has two keys – **churn and non-churn** – one for each population that will be presented. The value for each key is the collection of *Call_Failure* records for each population. Pay attention to the fact that, unlike a table data structure that has two dimensions (rows and columns), a dictionary only has one dimension.



In [ ]:
churn_possibilities = customer_df.Churn.unique()

box_sr = pd.Series('',index = churn_possibilities)

for poss in churn_possibilities:
    BM = customer_df.Churn == poss
    box_sr[poss] = customer_df[BM].Call_Failure.values

In [ ]:
print(box_sr)

In [ ]:
plt.figure(figsize=(5,3))

plt.boxplot(box_sr,vert=False)
plt.yticks([1,2],['Not Churn','Churn'])
plt.show()

> **Enumerate()** method adds a counter to an iterable and returns it in a form of enumerating object. This enumerated object can then be used directly for loops or converted into a list of tuples using the list() function.

In [ ]:
select_columns = ['Call_Failure', 'Subscription_Length', 'Seconds_of_Use',
                  'Frequency_of_use', 'Frequency_of_SMS', 'Distinct_Called_Numbers']
churn_possibilities = customer_df.Churn.unique()

plt.figure(figsize=(15,5))
for i,sc in enumerate(select_columns):
  for poss in churn_possibilities:
    BM = customer_df.Churn == poss
    box_sr[poss] = customer_df[BM][sc].values
  plt.subplot(2,3,i+1)  # subplot(nrows, ncols, index)
  plt.boxplot(box_sr,vert=False)
  plt.yticks([1,2],['Not Churn','Churn'])
  plt.title(sc)

plt.tight_layout()  # Adjust the padding between and around subplots.
plt.show()

## Example 2.3 – Data Cleaning Level Ⅰ and Ⅱ

In this example, we want to use **Electric Production.csv** to make predictions. We are specifically interested in being able to predict what the monthly electricity demand will be 1 month from now. This 1-month gap is designed in the prediction model so that the predictions that come from the model will have decision-making values; that is, the decision-makers will have time to react to the predicted value.

We would like to use **linear regression** to perform this prediction. The independent and dependent attributes for this prediction are shown in the following diagram:

<img src="https://drive.google.com/uc?id=151tanGbB1N9sa8ldRvMbG76YiFmMacoD" width="500"/>

1.   **Average demand of the month over the years**: For instance, if the month we want to predict demands for is March 2022, we want to use the average of the demands for every March in the previous years. So, we will collate the historical demands of March from the beginning of the data collection process (1985) to 2021 and calculate its average.
2.   **Slope of change for the demand of the month over the years**: For instance, if the month we want to predict demands for is March 2022, we want to use the slope of change in the demand in March over the years. As shown in the following diagram, we can fit a line on the Demand in March data points across the years. The slope of that fitted line will be used for prediction.
1.   **Average demands of months t-2, t-3, and t-4**: In the preceding diagram, the t, t-2, t-3, and t-4 notations are used to create a time reference. This time reference is that if we want to predict the demand of a month, we want to use the average demand of the following data points: the monthly demand of 2 months ago, the monthly demand of 3 months ago, and the monthly demand of 4 months ago. For instance, if we want to predict the monthly demand of March 2021, we'd want to calculate the average of January 2021, December 2020, and November 2020. Note that we skipped February 2021 as it was our planned decision-making gap.

<img src="https://drive.google.com/uc?id=1akVGn5q8snz_O4-gpISklstuwzBuVESA" width="500"/>

In [ ]:
month_df = pd.read_csv('Electric Production.csv')
month_df

### Level Ⅰ cleaning
---
The **month_df** dataset could do with the following level I data cleaning steps:
*   The title of the second column can be more intuitive.
*   The data type of the DATE column can be switched to datetime so that we can take advantage of datetime programming properties.
*   The default index that's been assigned to the data by pandas can be improved as the DATE column would provide a better and more unique identification.




In [ ]:
month_df.columns = ['Date','Demand']
month_df.set_index(pd.to_datetime(month_df.Date,format='%m/%d/%Y'),inplace=True)
month_df.drop(columns=['Date'],inplace=True)

In [ ]:
month_df

### Level Ⅱ cleaning

In [ ]:
attributes_dic={'IA1':'Average demand of the month',
                'IA2':'Slope of change for the demand of the month',
                'IA3': 'Average demands of months t-2, t-3 and t-4',
                'DA': 'Demand of month t'}

predict_df = pd.DataFrame(index=month_df.iloc[24:].index,columns=attributes_dic.keys())

In [ ]:
predict_df

**Why are the first 24 indices not included?** This is due to the second independent attribute: Slope of change for the demand of the month over the years. As the slope of demand change for each month will be needed for the described prediction model, we cannot have a meaningful slope value for the first 24 rows of month_df in predict_df. This is because we at least need two historical data points for each month to be able to calculate a slope for the second independent attribute.

### Fill out DA

In [ ]:
predict_df.DA = month_df.loc['1987-01-01':].Demand
predict_df

### Fill out IA1

To compute IA1, which is the <font color='blue'>Average demand of the month over the years</font>, we need to be able to filter month_df using the value of the month. To create such a capability, the following code maps a lambda function to month_df and extracts the month of each row:

In [ ]:
month_df['Month'] = list(map(lambda v:v.month, month_df.index))

In [ ]:
month_df

The function **ComputeIA1()** that is written to be applied to the rows of *predict_df*, performs the following steps:
1. First, it filters out *month_df* using the calculated *row_date* to remove the data points whose dates are after *row_date*.
2. Second, the function uses a Boolean mask to keep the data points with the same month as the row's month *(row_date.month)*.
3. Next, the function calculates the average demand of the filtered data points and then returns it.

In [ ]:
def ComputeIA1(r):
  row_date = r.name
  wdf = month_df.loc[:row_date].iloc[:-1]
  BM = wdf.Month == row_date.month
  return wdf[BM].Demand.mean()

predict_df.IA1 = predict_df.apply(ComputeIA1,axis=1)

In [ ]:
predict_df

### Fill out IA2

The **ComputeIA2()** function uses LinearRegression from sklearn.linear_model to find the fitted regression equation and then return the calculated coefficient of the model.

In [ ]:
from sklearn.linear_model import LinearRegression
def ComputeIA2(r):
  row_date = r.name
  wdf = month_df.loc[:row_date].iloc[:-1]
  BM = wdf.Month == row_date.month

  wdf = wdf[BM]
  wdf.reset_index(drop=True,inplace=True)
  wdf.drop(columns = ['Month'],inplace=True)
  wdf['integer'] = range(len(wdf))
  wdf['ones'] = 1

  lm = LinearRegression()
  lm.fit(wdf.drop(columns=['Demand']), wdf.Demand)
  return lm.coef_[0]

predict_df.IA2 = predict_df.apply(ComputeIA2,axis=1)

In [ ]:
predict_df

To understand the way ComputeIA2() finds the slope of change for each cell under predict_df.IA2, see the following screenshot, which shows the code and its output for calculating the slope for one cell under predict_df.IA2. The following screenshot calculates the IA2 value for the row with an index of 2017-10-01:

In [ ]:
row_date = '2017-10-01'
wdf = month_df.loc[:row_date].iloc[:-1]
BM = wdf.Month == 10
wdf = wdf[BM]
wdf.reset_index(drop=True,inplace=True)
wdf.drop(columns = ['Month'],inplace=True)
wdf['integer'] = range(len(wdf))
wdf['ones'] = 1

lm = LinearRegression()
lm.fit(wdf.drop(columns=['Demand']), wdf.Demand)
print('Slope = {}'.format(lm.coef_[0]))

wdf.plot.scatter(x='integer',y='Demand',marker='*',label='Data points',c='C0')

b = lm.intercept_
a = lm.coef_[0]

X = wdf.integer
y = b + a*X

plt.plot(X,y,label = 'Fitted regression',linestyle='--',c='C1')
plt.show()

# https://matplotlib.org/stable/tutorials/colors/colors.html

### Fill out IA3

The following code creates the **ComputeIA3()** function and applies it to *predict_df*. This function uses the index of *predict_df* to find the demand values from 2 months ago, 3 months ago, and 4 months ago.

It does this by filtering out all the data that is after *row_date* using <font color='red'>.loc[:row_date]</font>, and then by only keeping the fourth, third, and second rows of the remaining data from the bottom using <font color='red'>.iloc[-5:-2]</font>. Once the data filtering process is complete, the average of three demand values is returned:

In [ ]:
def ComputeIA3(r):
  row_date = r.name
  wdf = month_df.loc[:row_date].iloc[-5:-2]

  return wdf.Demand.mean()

predict_df.IA3 = predict_df.apply(ComputeIA3,axis=1)

In [ ]:
predict_df

### Doing the analytics – Using Linear Regression to create a predictive model

In [ ]:
from sklearn.linear_model import LinearRegression

X = predict_df.drop(columns=['DA'])
y = predict_df.DA

lm = LinearRegression()
lm.fit(X,y)

In [ ]:
print('intercept (b0) ', lm.intercept_)
coef_names = ['b1','b2','b3']
print(pd.DataFrame({'Predictor': X.columns,
                    'coefficient Name':coef_names,
                    'coefficient Value': lm.coef_}))

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(X.index, y, label='Actual')
plt.plot(X.index, lm.predict(X), label = 'Fitted', linestyle='--')
plt.legend()
plt.show()